# TSA Chapter 7: Case Study - VECM Estimation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch7/TSA_ch7_case_vecm/TSA_ch7_case_vecm.ipynb)

VECM(2) with coint_rank=1 fitted on FRED GS10 and TB3MS interest rates: error correction term, adjustment coefficients, and ECT vs changes.

In [ ]:
!pip install numpy pandas matplotlib statsmodels pandas_datareader yfinance -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.vector_ar.vecm import VECM
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm
import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']
plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})
def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
# Fetch real data from FRED
start, end = '2000-01-01', '2024-12-31'
gs10 = web.DataReader('GS10', 'fred', start, end).dropna()
tb3ms = web.DataReader('TB3MS', 'fred', start, end).dropna()
data = pd.DataFrame({'GS10': gs10['GS10'], 'TB3MS': tb3ms['TB3MS']}).dropna()
print(f'Period: {data.index[0].strftime("%Y-%m")} to {data.index[-1].strftime("%Y-%m")}, n={len(data)}')

# Fit VECM(2) with 1 cointegrating relation
vecm_model = VECM(data, k_ar_diff=2, coint_rank=1)
vecm_res = vecm_model.fit()
print(vecm_res.summary())

# Extract error correction term (ECT)
# ECT = beta' * y_{t-1} (cointegrating vector applied to lagged levels)
beta = vecm_res.beta
ect = data.values[:-1] @ beta  # ECT for each observation
ect = ect[2:]  # align with VECM residuals (k_ar_diff=2 removes 2 extra obs)
ect_series = pd.Series(ect.flatten(), index=data.index[3:])

# Adjustment coefficients (alpha)
alpha = vecm_res.alpha.flatten()

# Changes in rates (aligned with ECT)
d_gs10 = data['GS10'].diff().iloc[3:]
d_tb3ms = data['TB3MS'].diff().iloc[3:]

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Panel 1: ECT over time
axes[0].plot(ect_series.index, ect_series.values, color=GREEN, lw=0.8)
axes[0].axhline(0, color=GRAY, ls='--', lw=0.8)
axes[0].set_title('Error Correction Term (ECT)', fontweight='bold')
axes[0].set_ylabel('ECT')

# Panel 2: Alpha (adjustment coefficients) bar chart
bars = axes[1].bar(['GS10', 'TB3MS'], alpha, color=[BLUE, RED], width=0.5)
axes[1].axhline(0, color='black', lw=0.5)
for bar, val in zip(bars, alpha):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                f'{val:.4f}', ha='center', va='bottom' if val > 0 else 'top', fontsize=9)
axes[1].set_title('Adjustment Coefficients ($\\alpha$)', fontweight='bold')
axes[1].set_ylabel('$\\alpha$')

# Panel 3: ECT vs changes scatter
ect_aligned = ect_series.values
axes[2].scatter(ect_aligned, d_gs10.values, color=BLUE, alpha=0.3, s=10, label='$\\Delta$GS10')
axes[2].scatter(ect_aligned, d_tb3ms.values, color=RED, alpha=0.3, s=10, label='$\\Delta$TB3MS')
axes[2].axhline(0, color=GRAY, ls='--', lw=0.5)
axes[2].axvline(0, color=GRAY, ls='--', lw=0.5)
axes[2].set_xlabel('ECT (t-1)')
axes[2].set_ylabel('$\\Delta$ Rate')
axes[2].set_title('ECT vs Rate Changes', fontweight='bold')
axes[2].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False)

plt.tight_layout()
save_chart(fig, 'ch7_case_vecm')
plt.show()